# Abberation mapping workflow 2: Analyze zola results #

*26 Jan 2023 first refactoring*

*07 Sep 2023 updates/troubleshooting*

Originally by Debayan Saha.  Jamie White modified Debayan's notebooks. 

Updating for Jamie's environment on the cluster.

Documenting installation / file locations.

Cleaned-up version of workflow notebook.

## Load packages ##

In [1]:
import sys
sys.path.append(".")
# sys.path.append('/home/dsaha/.local/lib/python3.7/site-packages/')
from main import *
import warnings
warnings.filterwarnings(action='once')
import seaborn as sns
import pandas as pd
%load_ext autoreload
%autoreload 2
def min_max_norm(arr):
    return (arr - np.min(arr))/(np.max(arr) - np.min(arr))

In [2]:
%env PATH=/sw/apps/cuda/10.0.130/libnvvp:/sw/apps/cuda/10.0.130/extras/CUPTI/lib64/:/sw/apps/jupyter-tensorboard/0.1.6/bin:/sw/apps/hdf5/1.8.16/bin:/sw/apps/cuda/10.0.130/bin:/sw/bin:/usr/lib64/qt-3.3/bin:/sbin:/bin:/usr/sbin:/usr/bin
%env LD_LIBRARY_PATH=/sw/apps/hdf5/1.8.16/lib:/sw/apps/cuda/10.0.130/lib64:/sw/apps/cuda/10.0.130/lib:/sw/lib 
%env MODULEPATH=/sw/apps/modules/modulefiles:/usr/share/Modules/modulefiles:/etc/modulefiles:/lib/modules:/lib/modules/3.10.0-693.21.1.el7.x86_64

env: PATH=/sw/apps/cuda/10.0.130/libnvvp:/sw/apps/cuda/10.0.130/extras/CUPTI/lib64/:/sw/apps/jupyter-tensorboard/0.1.6/bin:/sw/apps/hdf5/1.8.16/bin:/sw/apps/cuda/10.0.130/bin:/sw/bin:/usr/lib64/qt-3.3/bin:/sbin:/bin:/usr/sbin:/usr/bin
env: LD_LIBRARY_PATH=/sw/apps/hdf5/1.8.16/lib:/sw/apps/cuda/10.0.130/lib64:/sw/apps/cuda/10.0.130/lib:/sw/lib
env: MODULEPATH=/sw/apps/modules/modulefiles:/usr/share/Modules/modulefiles:/etc/modulefiles:/lib/modules:/lib/modules/3.10.0-693.21.1.el7.x86_64


# Inputs

In [3]:
# dir
img_path        = "/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope/beads.tif"

In [4]:
# params
old_params = setup_param_file(img_path, load_mode=True)

unit     = old_params.get('unit') 
abb_list = old_params.get('abb_list')
save_dir = old_params.get('save_dir')
lam      = old_params.get('lam')

old_params

{'crop_shape': [40, 40, 40],
 'unit': [0.1, 0.1, 0.1],
 'abb_list': [3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'FIJI_MACRO_PATH': '/home/jwhite/projects/aberration_mapping/imageJ_macros/zola_macro_cluster_1p2_z_100_2ndfloor.ijm',
 'COMPUTERNAME': 'r02n14',
 'thresh_bck': 10,
 'lam': 0.515,
 'thresh_dist': 11.299999999999999,
 'save_dir': '/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope',
 'num_folders': 50}

## Check the Fiji macro in use

In [5]:
old_params['FIJI_MACRO_PATH']

'/home/jwhite/projects/aberration_mapping/imageJ_macros/zola_macro_cluster_1p2_z_100_2ndfloor.ijm'

## Check the save directory files

In [6]:
save_dir

'/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope'

In [7]:
! ls "{save_dir}"

abb_maps		      measurements	points
accepted_points_in_image.png  med_measurement	Results.csv
accepted_zola		      params.json	rmswe
amplitude_distribution.png    patches_for_zola	selected_widths.npy
beads.tif		      point_boxes	selected_widths.png
label_map.tif		      pointlist.npy


# Load zola and select the stable results

In [8]:
zola_full = load_and_select_zola_files(save_dir, lam, headless=True, show_plots=False)

0it [00:00, ?it/s]


In [11]:
zola_full

OrderedDict()

# Filter zola results by neighbors

In [10]:
print(f"Zola_files before filtering {len(zola_full)}")
med_del_keys = filter_files_by_med_dist_of_neighbors(zola_full)
print(f"Deleting {len(med_del_keys)} points because there medians values are far away from observed values")

# deleting the zola files
[zola_full.pop(x, None) for x in med_del_keys]
print(f"Final number of zola_files after filtering {len(zola_full)}")

Zola_files before filtering 0


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# Load corresponding crops

In [ ]:
plane_num = 10
crops = {key: tifffile.imread(f"{os.path.dirname(img_path)}/patches_for_zola/patches*/planes_{plane_num}_{key}.tif") for key in zola_full.keys()}
print(len(zola_full), len(crops))

# Show the accepted crops

In [ ]:
count = 0
for key,val in crops.items():
    count = count + 1
    print(key, np.max(val) - np.median(val))
    axs = show_patch(val)
    plt.plot(zola_full[key].values())
    plt.ylim(-0.2,0.2)
    plt.show()
    if count>50:
        break

# Plot the distribution

In [ ]:
vals = np.array([list(z.values()) for z in zola_full.values()])
colum_name = ['astig_x','astig_y', 'vert_y', 'coma_y', 'coma_x', 'trefoil_x', 'quadrafoil_x', 'sec_astig_x', 'spherical', 'sec_astig_y', 'quadrafoil_y']
p1 = pd.DataFrame(data=vals, columns=colum_name)
df_c = pd.melt(p1)

In [ ]:
# plt.style.use('dark_background')
fig,axes = plt.subplots(1,1,figsize=(15,7))
# g = sns.violinplot(data =df_c, x='variable',y='value', inner='quartile');
g = sns.swarmplot(data =df_c, x='variable',y='value');
plt.xticks(rotation=90, size=20)
plt.hlines(xmax=10.5, xmin=-0.5, y=0)
plt.xlabel("Zernike mode (ANSI order)", size=18)
plt.ylabel("Aberration amplitude in $\\mu$m",size=18)
plt.ylim(-0.3,0.3)
plt.yticks(size=20)
plt.tight_layout()
plt.show()
fig.savefig(f"{save_dir}/amplitude_distribution.png", transparent=True)

# Widths

In [ ]:
fwhm = get_xyz_widths(crops, unit=unit, headless=True)
print(f"Could measure {len(fwhm)} widths")

In [ ]:
# plot the histogram of widths
arr = np.array(list(fwhm.values()))
max_bin=5
n_bins = 20
bins = np.linspace(0,max_bin,n_bins);
colors = ('r','b','g')
names = ('FWHM_z','FWHM_y','FWHM_x')
for i,(a,c) in enumerate(zip(arr.T, colors)):
    sns.histplot(data=np.clip(a,bins[0], bins[-1]), bins=bins,kde=False, color=c, label=names[i]);
plt.legend() ;
plt.xlabel(r"FWHM in $\mu m$");
# plt.xlim(0.2,)
plt.savefig(f"{save_dir}/selected_widths.png")
np.save(f"{save_dir}/selected_widths.npy", arr)

# Aberration maps

In [ ]:
create_abb_map(img_path, zola_full, scale_factor=(4,4,4))

# RMSWE

In [ ]:
save_dir

In [ ]:
! ls "{save_dir}"

In [ ]:
find_rmswe(save_dir)